## Gaussian Discriminant Analysis

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [7]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [8]:
X.shape, y.shape

((768, 8), (768,))

In [9]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target

target_names = list(dataset.target_names)
print(target_names)

['setosa', 'versicolor', 'virginica']


In [10]:
# Change to binary class
y = (y > 0).astype(int)

In [11]:
np.unique(y)

array([0, 1])

In [12]:
class GaussianDiscAnalysis:    
    # Compute phi
    def compute_phi(self, y):
        return (1/len(y)) * len(y[y==1])
    
    def compute_mu(self, X, y, idx):
        return np.sum(X[y==idx], axis=0)/ len(y==idx)
    
    
    def compute_sigma(self, X, y):
        n = len(X)
        #y = y.reshape(-1,1)
        mu1 = self.mu[1]
        mu0 = self.mu[0]
        Xmu = X \
              - mu1*np.ones_like(X)*(y==1).reshape(-1,1) \
              - mu0*np.ones_like(X)*(y==0).reshape(-1,1)
        return (1/n) * Xmu.T@Xmu
    
    
    def compute_Pxyi(self, X, idx):
        """Probability of X given y"""
        m = X.shape[1]
        sigma_inv = np.linalg.inv(self.sigma)
        det_sigma = np.linalg.det(self.sigma)
        #mu_i = mu(X, y, idx)
        Pxi = (1/((2*np.pi)**(m/2))) \
                *(1/(det_sigma**0.5)) \
                * np.exp(- 0.5*np.sum(((X-self.mu[idx])@sigma_inv)*(X-self.mu[idx]), axis=1))
    #     Pxi = np.log(1) \
    #             - np.log((2*np.pi)**(m/2)) \
    #             - np.log(np.sqrt(det_sigma)) \
    #             - np.sum(((X-mu_i)@sigma_inv)*(X-mu_i), axis=1)
        return Pxi
    
    def fit(self, X, y):
        """Computes mean, covariance and proabilities of y (phi)"""
        self.mu = []
        for i in np.unique(y):
            self.mu.append(self.compute_mu(X, y, i))
        #self.mu1 = self.compute_mu(X, y, 1)
        self.sigma = self.compute_sigma(X, y)
        self.phi = self.compute_phi(y)
        
    def predict(self, X):
        Py0 = self.compute_Pxyi(X, 0) * (1-self.phi)
        Py1 = self.compute_Pxyi(X, 1) * self.phi
        Py0 = Py0.reshape(-1, 1)
        Py1 = Py1.reshape(-1, 1)
        return np.argmax(np.concatenate((Py0, Py1), axis=1), axis=1)
        

In [13]:
GDA = GaussianDiscAnalysis()

In [14]:
GDA.fit(X,y)

In [15]:
predictions = GDA.predict(X)

In [16]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [17]:
np.sum(predictions == y) / len(y)

1.0

In [357]:
class GaussianDiscAnalysis: 
    def compute_phi(self, y):
        n = len(y)
        phi = dict()
        for idx in range(len(self.classes)):
            phi[idx] = (1/n) * np.sum(y==idx)
        return phi
    
    def compute_mu(self, X, y):
        mu_dict = dict()
        for idx in range(len(self.classes)):
            # Add mu for each class
            mu_dict[idx] = np.sum(X[y==idx], axis=0)/ np.sum(y==idx)
        print(mu_dict)
        return mu_dict

    def compute_sigma(self, X, y):
        n = len(X)
        #y = y.reshape(-1,1)
        Xmu = X.copy()
        for idx in range(len(self.classes)):
            Xmu = Xmu \
              - self.mu[idx]*np.ones_like(Xmu)*(y==idx).reshape(-1,1)
        return (1/n) * Xmu.T@Xmu
    
    
    def compute_Pxyi(self, X, idx):
        """Probability of X given y"""
        m = X.shape[1]
        sigma_inv = np.linalg.inv(self.sigma)
        det_sigma = np.linalg.det(self.sigma)
        #mu_i = mu(X, y, idx)
        Pxi = (1/((2*np.pi)**(m/2))) \
                *(1/(det_sigma**0.5)) \
                * np.exp(- 0.5*np.sum(((X-self.mu[idx])@sigma_inv)*(X-self.mu[idx]), axis=1))
    #     Pxi = np.log(1) \
    #             - np.log((2*np.pi)**(m/2)) \
    #             - np.log(np.sqrt(det_sigma)) \
    #             - np.sum(((X-mu_i)@sigma_inv)*(X-mu_i), axis=1)
        return Pxi
    
    def fit(self, X, y):
        """Computes mean, covariance and proabilities of y (phi)"""
        self.classes = np.unique(y)
        self.mu = self.compute_mu(X, y)
        #self.mu0 = self.compute_mu(X, y, 0)
        #self.mu1 = self.compute_mu(X, y, 1)
        self.sigma = self.compute_sigma(X, y)
        self.phi = self.compute_phi(y)
        
    def predict_proba(self, X):
        n = len(X)
        Pyi = list()
        for idx in range(len(self.classes)):
            py_i = self.compute_Pxyi(X, idx) * self.phi[idx]
            Pyi.append(py_i)
        return np.squeeze(np.array(Pyi)).reshape(n, -1)
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
        #Py0 = self.compute_Pxyi(X, 0) * (1-phi(y))
        #Py1 = self.compute_Pxyi(X, 1) * phi(y)
        #Py0 = Py0.reshape(-1, 1)
        #Py1 = Py1.reshape(-1, 1)
        #return np.argmax(np.concatenate((Py0, Py1), axis=1), axis=1)

In [341]:
np.unique([2,4,5,2])

array([2, 4, 5])